In [1]:
# Import all needed libraries and sublibraries

import tensorflow as tf

import numpy as np 
    
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


import pandas as pd

import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import scale
from sklearn.metrics import mean_squared_error

from matplotlib import pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
dataframe = pd.read_csv("aafc_data.csv")  # importing the dataset
# summarize shape
print(dataframe.shape)
print("\n")
# summarize first few lines
dataframe.head()

(42007, 166)




,Unnamed: 0,TWP_ID,ECODISTRICT_ID,YEAR,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,0,00101E1,852.0,2010,867.766846,53.6,111.1,109.7,117.9,46.4,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,1,00101W1,852.0,2010,673.685028,57.2,114.7,110.5,114.0,46.2,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,2,00101W2,796.0,2010,824.303864,39.0,96.4,109.8,101.2,111.4,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,3,00102E1,853.0,2010,1006.708496,37.5,158.2,157.8,161.4,46.9,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,4,00102W1,852.0,2010,869.040283,57.2,114.7,110.5,114.0,46.2,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


In [3]:
# dropping those columns that are not useful
dataframe.drop(['Unnamed: 0', 'TWP_ID', 'YEAR'], axis=1, inplace=True)

In [4]:
# looking at the dataset after dropping the columns 
dataframe.head()

,ECODISTRICT_ID,YieldKgAcre,SumPcpn18_20,SumPcpn19_21,SumPcpn20_22,SumPcpn21_23,SumPcpn22_24,SumPcpn23_25,SumPcpn24_26,SumPcpn25_27,...,SoilMoisture29_31,SoilMoisture30_32,SoilMoisture31_33,SoilMoisture32_34,SoilMoisture33_35,SoilMoisture34_36,SoilMoisture35_37,SoilMoisture36_38,SoilMoisture37_39,SoilMoisture38_40
0,852.0,867.766846,53.6,111.1,109.7,117.9,46.4,69.3,60.0,44.6,...,16.960125,18.766207,17.186998,15.461519,19.738222,22.958089,27.206203,26.480087,28.678156,26.308484
1,852.0,673.685028,57.2,114.7,110.5,114.0,46.2,68.1,55.9,34.9,...,16.328520,17.926029,16.787544,14.779726,20.245149,23.608204,28.560990,27.324254,29.079177,26.927224
2,796.0,824.303864,39.0,96.4,109.8,101.2,111.4,153.0,163.6,98.8,...,13.117879,12.869142,12.831834,14.126196,16.385776,18.650751,20.287069,20.514132,19.564788,16.681692
3,853.0,1006.708496,37.5,158.2,157.8,161.4,46.9,79.5,67.5,40.4,...,17.060778,18.699156,17.345822,15.998957,20.091525,22.761273,26.337430,25.559602,27.611729,25.575794
4,852.0,869.040283,57.2,114.7,110.5,114.0,46.2,68.1,55.9,34.9,...,16.050993,17.556860,16.612026,14.480150,20.467884,23.893858,29.156274,27.695178,29.255386,27.199097


In [14]:
eco_district_ids_list = [748, 826, 752, 745, 808, 792, 849, 729, 753, 709, 782,825,760,767,794,749,816,830,756,765,770,773,707,724,795,736,763,780,832,723,783,696,733,822,754,687,758,766,
                          726,717,701,705,850,755,803,706,813,741,789,841,852,702,735,375,689,714,839,693,711,694,784,820,821,831,751,680,704,
                          757,817,838,715,747,776,807,690, 697,695,698,710,796,809,840,854,685,700,771,805,810,824,844,734,762,764,774,785,851,775,716,739,843,661,682,847,669,815,846,
       772,837,742,761,853,672,699,768,778,657,677,691,743,759,827,660,819,848,652,718,686,720, 811,855,379,647,659,662,668,833,834]

In [15]:
msetest = []
msetrain = []
mae = []
acc = []
for i in eco_district_ids_list:
    df1 = dataframe[dataframe['ECODISTRICT_ID']==i]   # created a dataframe for one of the eco district
    df1.drop(['ECODISTRICT_ID'], axis=1, inplace=True)  # dropped the 'ECODISTRICT_ID' column before modeling

    # split data into X and y
    x = pd.DataFrame(df1.drop(labels=['YieldKgAcre'], axis=1)) # x contains the predictors (not the target value 'YieldKgAcre')
    y = pd.DataFrame(df1['YieldKgAcre'])                       # y contains the dependent variable ('YieldKgAcre').
    
    
    # splitting the x and y datasets to train and test with the ratio 80:20 ratio.
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

# Run standardization on X variables
# So that it centers around 0 and has standard deviation of 1.
    X_train = scale(X_train)
    X_test = scale(X_test)
    
    
    
    # Defines "deep" model and its structure
    model = Sequential()
    model.add(Dense(15, input_shape=(161,), activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(15, activation='relu'))    
    model.add(Dense(1,))
    model.compile(Adam(lr=0.003), 'mean_squared_error')

# Pass several parameters to 'EarlyStopping' function and assigns it to 'earlystopper'
    earlystopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=15, verbose=1, mode='auto')

# Fits model over 2000 iterations with 'earlystopper' callback, and assigns it to history
    history = model.fit(X_train, y_train, epochs = 2000, validation_split = 0.2,shuffle = True, verbose = 0, 
                    callbacks = [earlystopper])

# Plots 'history'
# history_dict=history.history
# loss_values = history_dict['loss']
# val_loss_values=history_dict['val_loss']
# plt.plot(loss_values,'bo',label='training loss')
# plt.plot(val_loss_values,'r',label='training loss val')

# Runs model with its current weights on the training and testing data
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    msetest.append(np.sqrt(mean_squared_error(y_test,y_test_pred)))
    msetrain.append(np.sqrt(mean_squared_error(y_train,y_train_pred)))
# Calculates and prints r2 score of training and testing data
    print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred)))
    print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_test_pred)))
    
    # Calculate the absolute errors
    errors = abs(y_test_pred - y_test)

#  # Print out the mean absolute error (mae)
# print('Mean Absolute Error:', np.mean(errors), 'degrees.')
    mae.append(np.mean(errors))


# Calculate mean absolute percentage error (MAPE)
    mape = 100 * (errors / y_test)

# Calculate and display the accuracy
    accuracy = 100 - np.mean(mape)
    #print('Accuracy:', round(accuracy, 2), '%.')
    acc.append(round(accuracy, 2))

C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 78: early stopping
8/8 [==============================] - 0s 2ms/step
The R2 score on the Train set is:	0.828
The R2 score on the Test set is:	0.685


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 94: early stopping
8/8 [==============================] - 0s 867us/step
The R2 score on the Train set is:	0.778
The R2 score on the Test set is:	0.656


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


...

Epoch 56: early stopping
1/1 [==============================] - 0s 0s/step
The R2 score on the Train set is:	0.563
The R2 score on the Test set is:	-0.684


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 65: early stopping
1/1 [==============================] - 0s 16ms/step
The R2 score on the Train set is:	-2.464
The R2 score on the Test set is:	-3.344


C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
C:\Users\Mehul Bhargava\miniconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 73: early stopping
1/1 [==============================] - 0s 14ms/step
The R2 score on the Train set is:	-2.042
The R2 score on the Test set is:	-3.874


In [16]:
import re
accn=[]
for i in range(len(acc)):
    acc[i] = acc[i].to_string()
    sentence = acc[i]
    s = [float(s) for s in re.findall('[0-9.]+', sentence)]
    accn.append(s)

In [17]:
maen=[]
for i in range(len(mae)):
    mae[i] = mae[i].to_string()
    sentence = mae[i]
    s = [float(s) for s in re.findall('[0-9.]+', sentence)]
    maen.append(s)

In [18]:
mae = [item for sublist in maen for item in sublist]
acc = [item for sublist in accn for item in sublist]

In [19]:
results = pd.DataFrame(list(zip(msetrain, msetest, mae, acc)),
               columns =['MSE Train', 'MSE Test', 'Mean Absolute Error', 'Accuracy'])

In [20]:
results

,MSE Train,MSE Test,Mean Absolute Error,Accuracy
0,82.192957,106.526173,81.033360,88.22
1,97.001202,117.610131,92.964331,85.39
2,78.149590,100.640085,81.934493,88.95
3,68.416987,84.425914,68.557577,91.25
4,90.807383,124.862709,97.359612,87.01
...,...,...,...,...
132,270.290753,577.747879,479.535848,38.50
133,148.982364,521.164764,514.949656,29.05
134,173.893685,1192.597645,810.597130,34.69
135,231.320104,525.436019,428.470683,43.60


In [21]:
results["Accuracy"].mean()

80.72766423357663

In [22]:
results.to_csv('neural_net_results.csv')